In [23]:
%tensorflow_version 2.x

In [24]:
import tensorflow as tf
import numpy as np
import os
from keras.datasets import imdb
from keras.preprocessing import sequence

In [25]:
vocabSize = 88584

maxLen = 250
batchSize = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= vocabSize)

In [ ]:
train_data[0]

In [28]:
#to make the review of the same length . i.e padding

train_data = sequence.pad_sequences(train_data,maxLen)
test_data = sequence.pad_sequences(test_data, maxLen)


In [29]:
# creating the model 
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocabSize, 32), tf.keras.layers.LSTM(32), tf.keras.layers.Dense(1,activation = "sigmoid")])

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          2834688   
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [58]:
#training the model 
model.compile(loss="binary_crossentropy", optimizer= 'rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs = 5, validation_split = 0.2)

Epoch 1/5
625/625 [==============================] - 36s 55ms/step - loss: 0.1854 - acc: 0.9337 - val_loss: 0.3360 - val_acc: 0.8846
Epoch 2/5
625/625 [==============================] - 33s 54ms/step - loss: 0.1555 - acc: 0.9434 - val_loss: 0.2890 - val_acc: 0.8986
Epoch 3/5
625/625 [==============================] - 34s 54ms/step - loss: 0.1313 - acc: 0.9545 - val_loss: 0.2964 - val_acc: 0.8900
Epoch 4/5
625/625 [==============================] - 34s 54ms/step - loss: 0.1137 - acc: 0.9615 - val_loss: 0.3405 - val_acc: 0.8914
Epoch 5/5
625/625 [==============================] - 34s 54ms/step - loss: 0.0985 - acc: 0.9665 - val_loss: 0.3593 - val_acc: 0.8826


In [59]:
results = model.evaluate(test_data, test_labels)

print(results)

782/782 [==============================] - 13s 16ms/step - loss: 0.4620 - acc: 0.8469
[0.4620412588119507, 0.8468800187110901]


In [60]:
import keras.preprocessing
# encode the text to the format the model can understand 

word_index = imdb.get_word_index()

def encode_text(text) :
  tokens =keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], maxLen)[0]


In [61]:
text = "OMG that movie was just fabulous"
encoded = encode_text(text)
print(encoded)

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [62]:
# decode function go from integer to wrds
reverse_word_index = {value:key for (key,value) in word_index.items()}

def decode_int(integers):
  pad =0
  text=""
  for num in integers:
    if num!=pad:
      text +=  reverse_word_index[num] + " "

  return text[:-1]

print(decode_int(encoded))
    
  




omg that movie was just fabulous


In [63]:
# making prediction

def pred(text):
  encoded_text= encode_text(text)
  preds = np.zeros((1,250))
  preds[0] = encoded_text
  results = model.predict(preds)
  print(results[0])

pos_review = "that just a fine piece of art the actors are amazing"
pred(pos_review)

neg_review = "i hated the movie so much and i would not watch it anymore "
pred(neg_review)

[0.68634415]
[0.9482574]
